# Testing with R

In [1]:
install.packages("sparklyr")

Installing package into ‘/usr/local/spark-3.0.1-bin-hadoop3.2/R/lib’
(as ‘lib’ is unspecified)



In [2]:
install.packages("tidyverse")

Installing package into ‘/usr/local/spark-3.0.1-bin-hadoop3.2/R/lib’
(as ‘lib’ is unspecified)



In [3]:
install.packages("nycflights13")

Installing package into ‘/usr/local/spark-3.0.1-bin-hadoop3.2/R/lib’
(as ‘lib’ is unspecified)



### Import Libraries

In [4]:
library(sparklyr)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ purrr::invoke() masks sparklyr::invoke()
✖ dplyr::lag()    masks stats::lag()



In [5]:
conf <- spark_config()

In [6]:
conf$`sparklyr.cores.local` <- 6
conf$`sparklyr.shell.driver-memory` <- "12G"

In [7]:
sc <- spark_connect(master = "local", config = conf)

### Load Parquet Frames

In [8]:
system("ls data/*", intern=TRUE)

[1] "data/s3_parquet_logs.c000"

In [9]:
txt <- spark_read_parquet(sc, "logdata", "data/*")

### Build UDFs

In [10]:
test <- "there/are/a/lot/of/words"
dplyr::last(str_split(test, '/', simplify=TRUE))

[1] "words"

In [11]:
get_object <- function(string) {
    splitted <- str_split(string, '/', simplify=TRUE)
    s3_object <- dplyr::last(splitted)
    return(s3_object)
}

In [12]:
get_object(test)

[1] "words"

In [13]:
splitted <- str_split(test, '/', simplify=TRUE)
paste(splitted[,-ncol(splitted)], collapse="/")

[1] "there/are/a/lot/of"

In [14]:
get_prefix <- function(string) {
    
    splitted <- stringr::str_split(string, '/', simplify=TRUE)
    all_but_last <- splitted[,-ncol(splitted)]
    concat <- paste(all_but_last, collapse = "/")
    return(concat)
}

In [15]:
get_prefix(test)

[1] "there/are/a/lot/of"

In [39]:
# new attempt at R function for spark apply
split_key <- function(df, colName) {
    
    splitted <- stringr::str_split(df[[colName]], '/', simplify=TRUE)
    all_but_last <- splitted[,-ncol(splitted)]
    concat <- paste(all_but_last, collapse = "/")
    object <- dplyr::last(splitted)
    
    df %>% mutate(prefix = concat, object = object)
}

### Process Dataframes

In [42]:
txt2 <- txt %>%
            filter(instr(key, "dmp")>0)
txt2

# Source: spark<?> [?? x 28]
   bucketowner bucket requestdatetime remoteip requester requestid operation
   <chr>       <chr>  <chr>           <chr>    <chr>     <chr>     <chr>    
 1 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… 15F94453… REST.HEA…
 2 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… DC0D21AA… REST.HEA…
 3 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… 637FF3AD… REST.GET…
 4 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… DEC9747E… REST.HEA…
 5 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… A0B7D291… REST.HEA…
 6 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… E916FC6B… REST.HEA…
 7 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… A558C5A8… REST.GET…
 8 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… E9E87B55… REST.GET…
 9 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… 234D569D… REST.HEA…
10 7409ed49be… alpha… 13/Jun/2020:04… 10.121.… arn:aws:… EB35C9B6… REST.GET…
# … with more rows, and 21 more variables: key 

In [66]:
# doesn't work
#spark_apply(txt2, split_key, context = { colName <- "key" })
variable = "test"
as_label(variable)

[1] "\"test\""

In [70]:
# so string split is returning a list object which spark doesn't know how to deal with
get_object_spark <- function(df, col, newColName) {
    df %>%
      mutate( as_label(newColName) <- reverse(split(reverse( df[[col]] ),'/')[0]) )
} 

In [91]:
# We can just hive regex to get the last thing
txt2 %>% 
    select(key, requestid, operation) %>%
    mutate(object := regexp_extract(key, '.*/([^/]+)$', 1),
          prefix := regexp_extract(key, '.*/([^/]+)$', 0))
#sdf_separate_column("object") 

# sdf_separate_column(object, 1)

# Source: spark<?> [?? x 5]
   key                 requestid   operation  object         prefix             
   <chr>               <chr>       <chr>      <chr>          <chr>              
 1 dmp/rawcleansed/ed… 15F944538A… REST.HEAD… part-00123-8a… dmp/rawcleansed/ed…
 2 dmp/edh_fs_config/… DC0D21AA4F… REST.HEAD… part-00000-33… dmp/edh_fs_config/…
 3 dmp/rawcleansed/ed… 637FF3AD75… REST.GET.… part-00010-86… dmp/rawcleansed/ed…
 4 dmp/edh_fs_config/… DEC9747EEC… REST.HEAD… part-00000-33… dmp/edh_fs_config/…
 5 dmp/rawcleansed/ed… A0B7D2916D… REST.HEAD… part-00054-f9… dmp/rawcleansed/ed…
 6 dmp/edh_fs_config/… E916FC6B86… REST.HEAD… part-00000-33… dmp/edh_fs_config/…
 7 dmp/rawcleansed/ed… A558C5A885… REST.GET.… part-00191-42… dmp/rawcleansed/ed…
 8 dmp/edh_fs_config/… E9E87B55F7… REST.GET.… part-00000-33… dmp/edh_fs_config/…
 9 dmp/rawcleansed/ed… 234D569DE2… REST.HEAD… part-00156-44… dmp/rawcleansed/ed…
10 dmp/edh_fs_config/… EB35C9B672… REST.GET.… part-00000-34… dmp/edh_fs_config/…
